In [195]:
import pandas as pd

In [100]:
df = pd.read_parquet("submissions.parquet")

In [197]:
df = pd.read_parquet("labeled_post.parquet")

In [220]:
len(df)

80571

In [198]:
df.head()

,link_flair_text,num_comments,over_18,score,url,selftext,title,id,edited,is_self,permalink,downs,ups,created,date
0,Not the A-hole,1155,False,9597,https://www.reddit.com/r/AmItheAsshole/comments/1284npp/aita_for_embarrassing_my_husband_in_front_of_his/,"I (26 F) am married to K (25 M) , i met him a few year back when my best friend and his best friend started dating, after that our friend groups kind of joined together. So all of my husband’s friends are also my friends and all my friends are his friends.\n\nI’ve know my best friend (A) for 15 years, we’ve always been extremely close and lived together at one point, she’s literally part of my family, she comes to all of my family events, my siblings refer to her as their sister and she just been extremely involved in my life for years.\nNow, me and my husband have a 6 month old daughter. My dad is extremely wealthy so he helps us out financially until my daughter is old enough to go to school, then i will take a job at his company that pays well. My husband has a job that he works from 7am - 4 pm and then afterwards him and his friends often hangout at my house or at one of their house’s. \n\nHere’s the problem, my husband’s best friend cheated on my best friend a few weeks ago, she’s completely heartbroken and since they lived together my husband and i agreed that it’s okay that she stays with us until they work things out. She’s not really up to seeing him just yet. A is a huge help with the baby, she’s so good with her and the baby loves her. When my husband and his friends get home they normally talk until 8 pm ish so he doesn’t help with the baby too much when he gets home, which i don’t mind it’s just nice to have a little help. Yesterday night my husband opened the front door , peaked his head in, looked right at me and A , rolled his eyes and walked out. Him and his friends came in a few moments later and were all being cold and rude to A all evening. After an hour or two my husband walks right up to me and A and says “does she always have to be here? I want to bring (friends name) here tonight.” And rolled his eyes, i responded “if i want to bring my best friend into the house my dad paid for, i can, and if i choose to have somebody help me with the baby that you’re not looking after, it’s going to be her.” His friends just sat there silently and he just walked away and sat back down . He’s giving me the silent treatment and A left and is staying with her mom as she feels uncomfortable being in the house now. So AITA?",AITA for embarrassing my husband in front of his friends?,1284npp,0.000000e+00,True,/r/AmItheAsshole/comments/1284npp/aita_for_embarrassing_my_husband_in_front_of_his/,0.0,9597,1.680307e+09,2023-04-01
1,Not the A-hole,19,False,8,https://www.reddit.com/r/AmItheAsshole/comments/1284q84/aita_from_eating_a_cookie/,"Story is just as stupid as the title.\n\nSo my sister (25) bough cookies home last night. They were $5 for a 8 pack. Well she didn't say that they weren't for everyone amd since she frequently goes grocery shopping for my mom I thought they were for everyone so I ate one today. She goes and asks my little sister (14) if she had them, she says no so my sister comes in my room while I'm trying to sleep (the box for the cookies were in my trash can because there was one left and I'm not dirtying a dish if i don't have to. Well she sees it amd takes it out of the trash and throws at me which woke me up. She tells me that she wants me to pay for the whole box because her kid is sad she didn't get the last cookie. I said that I'd pay for the single cookie I took she refused. So she goes and says she's not leaving my room until I give her $5 so I call my mom and my mom tells her to get out of my room. She ends up saying that she'll wait until I go to school on monday (I'm 16 so still in hs) so she can steal my stuff to pay for the box (she's done this before when I didn't know that the ice cream in the freezer was hers and she stole my $100 hair straigh

In [199]:
df.selftext

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               I (26 F) am married to K (25 M) , i met him a few year back when my best friend and his best friend started dating, after that our friend groups kind of joined together. So all of my husband’s friends are also my fri

In [200]:
df.head(1).style.set_table_styles([dict(selector='td', props='max-width: 600px;'),])

,link_flair_text,num_comments,over_18,score,url,selftext,title,id,edited,is_self,permalink,downs,ups,created,date
0,Not the A-hole,1155,False,9597,https://www.reddit.com/r/AmItheAsshole/comments/1284npp/aita_for_embarrassing_my_husband_in_front_of_his/,"I (26 F) am married to K (25 M) , i met him a few year back when my best friend and his best friend started dating, after that our friend groups kind of joined together. So all of my husband’s friends are also my friends and all my friends are his friends. I’ve know my best friend (A) for 15 years, we’ve always been extremely close and lived together at one point, she’s literally part of my family, she comes to all of my family events, my siblings refer to her as their sister and she just been extremely involved in my life for years. Now, me and my husband have a 6 month old daughter. My dad is extremely wealthy so he helps us out financially until my daughter is old enough to go to school, then i will take a job at his company that pays well. My husband has a job that he works from 7am - 4 pm and then afterwards him and his friends often hangout at my house or at one of their house’s. Here’s the problem, my husband’s best friend cheated on my best friend a few weeks ago, she’s completely heartbroken and since they lived together my husband and i agreed that it’s okay that she stays with us until they work things out. She’s not really up to seeing him just yet. A is a huge help with the baby, she’s so good with her and the baby loves her. When my husband and his friends get home they normally talk until 8 pm ish so he doesn’t help with the baby too much when he gets home, which i don’t mind it’s just nice to have a little help. Yesterday night my husband opened the front door , peaked his head in, looked right at me and A , rolled his eyes and walked out. Him and his friends came in a few moments later and were all being cold and rude to A all evening. After an hour or two my husband walks right up to me and A and says “does she always have to be here? I want to bring (friends name) here tonight.” And rolled his eyes, i responded “if i want to bring my best friend into the house my dad paid for, i can, and if i choose to have somebody help me with the baby that you’re not looking after, it’s going to be her.” His friends just sat there silently and he just walked away and sat back down . He’s giving me the silent treatment and A left and is staying with her mom as she feels uncomfortable being in the house now. So AITA?",AITA for embarrassing my husband in front of his friends?,1284npp,0.000000,True,/r/AmItheAsshole/comments/1284npp/aita_for_embarrassing_my_husband_in_front_of_his/,0.000000,9597,1680307357.000000,2023-04-01


In [201]:
import re

# age & gender symbols (i'm 20F, me (30M), etc.)
P = re.compile(r'\b(I|me|I\'m|im|am|my|myself)\s(\d{2}\s?(F|M)|(F|M)\s?\d{2})\b', re.IGNORECASE)

# age & gender symbols for title ([20F], [M 30])
P_title = re.compile(r'((\[\d{2}\s?(F|M)\])|(\[(F|M)\s?\d{2}\]))', re.IGNORECASE)

# age only
P_age = re.compile(r'\b(I|me|I\'m|im|am|myself)\s(\d{2})\b', re.IGNORECASE)
P_age_title = re.compile(r'\b(I|me|I\'m|im|am|myself)\s(\d{2})\b', re.IGNORECASE)

# gener only
P_gender_self = re.compile(r'\b(I|me|I\'m|im|am)\s(a\s)?(female|male|woman|man|mother|mom|dad|father)\b', re.IGNORECASE)
P_gender_other = re.compile(r'\b(my)\s(ex-)?(wife|husband|girlfriend|boyfriend|gf|bf)\b', re.IGNORECASE)

gender_self = {
    "female": "F",
    "male": "M",
    "woman": "F",
    "man": "M",
    "mother": "F",
    "father": "M",
    "mom": "F",
    "dad": "M",
}

gender_other = {
    "wife": "M",
    "girlfriend": "M",
    "gf": "M",
    "husband": "F",
    "boyfriend": "F",
    "bf": "F",
}

def clean(text):
    clean_text = text.lower()
    clean_text = re.sub(r"[()/,.\s]", " ", clean_text)
    clean_text = re.sub(r"\s{2,}", " ", clean_text)
    return clean_text


def extract_age_gender(text):

    clean_text = clean(text)
    
    age = None
    gender = None

    age_gender_match = P.findall(clean_text)
    if age_gender_match:
        selfref = age_gender_match[0][0]
        symbol = age_gender_match[0][1]
        age = int("".join([char for char in symbol if char.isnumeric()]))
        gender = [char for char in symbol if char.isalpha()][0].upper()
    else:
        age_match = P_age.findall(clean_text)
        if age_match:
            age = int(age_match[0][1])

        gender_match = P_gender_self.findall(clean_text)
        if gender_match:
            gender = gender_self[gender_match[0][2]]
        else:
            gender_match = P_gender_other.findall(clean_text)
            if gender_match:
                gender = gender_other[gender_match[0][2]]

    return age, gender


def extract_age_gender_from_title(text):

    clean_text = clean(text)
    age = None
    gender = None

    age_gender_match = P.findall(clean_text)
    if age_gender_match:
        selfref = age_gender_match[0][0]
        symbol = age_gender_match[0][1]
        age = int("".join([char for char in symbol if char.isnumeric()]))
        gender = [char for char in symbol if char.isalpha()][0].upper()
    elif age_gender_match := P_title.findall(clean_text):
        # print(age_gender_match)
        symbol = age_gender_match[0][0]
        age = int("".join([char for char in symbol if char.isnumeric()]))
        gender = [char for char in symbol if char.isalpha()][0].upper()
    else:
        age_match = P_age.findall(clean_text)
        if age_match:
            age = int(age_match[0][1])
        else:
            age_match = P_age_title.findall(clean_text)
            if age_match:
                age = int(age_match[0][0])
        
        gender_match = P_gender_self.findall(clean_text)
        if gender_match:
            gender = gender_self[gender_match[0][2]]
        else:
            gender_match = P_gender_other.findall(clean_text)
            if gender_match:
                gender = gender_other[gender_match[0][2]]

    return age, gender

In [202]:
titles = [
    "I'm 17",
    "[17F]",
    "[M 30]",
    "(20M)",
    "I'm (20M)",
    "17 years old",
    "[female]",
    "male",
    "[17]",
    "20",
    "GF (20) and I (21)",
]

for text in titles:
    print(text)
    print(extract_age_gender_from_title(text))
    print()

I'm 17
(17, None)

[17F]
(17, 'F')

[M 30]
(30, 'M')

(20M)
(None, None)

I'm (20M)
(20, 'M')

17 years old
(None, None)

[female]
(None, None)

male
(None, None)

[17]
(None, None)

20
(None, None)

GF (20) and I (21)
(21, None)



In [203]:
texts = [
    "I'm",
    "Im",
    "I'm 17 years old",
    "My 17 years old son",
    "im 17 years old",
    "My girlfriend",
    "My ex-wife",
    "I'm mother",
    "I'm a dad",
    "GF (20) and I (21)",
    "my (27/f) and my husband",
    "my (f/27) and my husband",
    "I am a dad of 3 (10F, 10M, and 6F). My 10 year old daughter is pretty girly but my 6 year old daughter follows",
    "My parents recently told me (22f) that they had considered another name for me when mom was pregnant with me.",
    "My F56 husband M55 insists I get up with him everyday. I set the alarm for 330 am every day he works, get up,",
    "My F56 husband M55 insists",
    "My F 56 husband M55 insists",
    "My 56 F husband M55 insists",
    "My 56f husband M55 insists",
    "My f56 husband M55 insists",
    "My son f56 husband M55 insists",
    "I(24) have siblings (4f, 13f, and 19f), and they have lived with me for 5 years. And our mother is not mentally there.",
    "I've been with my girlfriend for just over a year. We've been living together since February. She has a 14 year old daughter from her previous marriage.",
    "I (25F) has been married to my SO (29M) for 2 years.Currently 8M pregnant.I work in IT whereas my SO work in pharmacy. ",
    "Yesterday my friends and I were hanging out in the park: me + my dog, my friend Cathy + her dog, and another friend Jenny.",
    "I (33F) met my brother (30M)'s girlfriend (29F) at a family dinner last night, they've been dating for six months and now things are getting slightly more serious he wanted her to meet me and our parents. ",
    "My ex-wife, Lena, and I have a son, Matt (6M) and I have 50/50 custody. We don't have such a... healthy co-parenting, we avoid each other most of the time and everything in relation",
    "I, 17F, am graduating high school next month and am set to attend my first-choice college with a partial scholarship in the fall. It's an instate school about an hour away, ",
    "First of all i'm sorry for my bad english. All names are fake for obvious reasons. Some background: In march of 2019 my(26m) mother(48f) di*d from heart attack in sleep. That was huge blow for my family... especialy for my father(55m) after 30y together",
]

for text in texts:
    print(text)
    print(extract_age_gender(text))
    print()

I'm
(None, None)

Im
(None, None)

I'm 17 years old
(17, None)

My 17 years old son
(None, None)

im 17 years old
(17, None)

My girlfriend
(None, 'M')

My ex-wife
(None, 'M')

I'm mother
(None, 'F')

I'm a dad
(None, 'M')

GF (20) and I (21)
(21, None)

my (27/f) and my husband
(27, 'F')

my (f/27) and my husband
(27, 'F')

I am a dad of 3 (10F, 10M, and 6F). My 10 year old daughter is pretty girly but my 6 year old daughter follows
(None, 'M')

My parents recently told me (22f) that they had considered another name for me when mom was pregnant with me.
(22, 'F')

My F56 husband M55 insists I get up with him everyday. I set the alarm for 330 am every day he works, get up,
(56, 'F')

My F56 husband M55 insists
(56, 'F')

My F 56 husband M55 insists
(56, 'F')

My 56 F husband M55 insists
(56, 'F')

My 56f husband M55 insists
(56, 'F')

My f56 husband M55 insists
(56, 'F')

My son f56 husband M55 insists
(None, None)

I(24) have siblings (4f, 13f, and 19f), and they have lived with me fo

In [227]:
flair_list = [
 'Not the A-hole',
 'Asshole',
 'No A-holes here',
 'Everyone Sucks',
 'Not enough info',
 # 'UPDATE',
 # 'TL;DR',
 'not the a-hole',
 # 'POO Mode Activated 💩',
 'asshole',
 # '',
 # 'META',
 'Shitpost',
 'not the asshole',
 'no a--holes here',
 'everyone sucks',
 'too close to call',
 'not enough info']

In [228]:
df_c = df.loc[df.link_flair_text.isin(flair_list)].copy()

In [229]:
%%time
df_c['age_gender_title'] = df_c.title.apply(lambda x: extract_age_gender_from_title(x))

CPU times: user 1.64 s, sys: 51.9 ms, total: 1.69 s
Wall time: 1.91 s


In [230]:
%%time
df_c['age_gender'] = df_c.selftext.apply(lambda x: extract_age_gender(x))

CPU times: user 24.8 s, sys: 351 ms, total: 25.2 s
Wall time: 25.8 s


In [231]:
df_c['age_title'] = df_c.age_gender_title.apply(lambda x: x[0])
df_c['gender_title'] = df_c.age_gender_title.apply(lambda x: x[1])

In [232]:
df_c['age'] = df_c.age_gender.apply(lambda x: x[0])
df_c['gender'] = df_c.age_gender.apply(lambda x: x[1])

In [233]:
df_c.columns

Index(['link_flair_text', 'num_comments', 'over_18', 'score', 'url',
       'selftext', 'title', 'id', 'edited', 'is_self', 'permalink', 'downs',
       'ups', 'created', 'date', 'age_gender_title', 'age_gender', 'age_title',
       'gender_title', 'age', 'gender'],
      dtype='object')

In [234]:
# 判定できた割合
df_c[['age_title', 'age', 'gender_title', 'gender']].notnull().sum() / df_c.shape[0] * 100.00

age_title        1.255433
age             44.978269
gender_title    12.934310
gender          56.992425
dtype: float64

In [235]:
# 年齢 本文で判定できなかったがタイトルで判定できたもの
len(df_c.loc[df_c.age_title.notnull() & df_c.age.isnull()]) / df_c.shape[0] * 100.00

0.9698249099714392

In [236]:
# 性別 本文で判定できなかったがタイトルで判定できたもの
len(df_c.loc[df_c.gender_title.notnull() & df_c.gender.isnull()]) / df_c.shape[0] * 100.00

1.7744939773997266

In [237]:
pd.set_option('display.max_colwidth', None)

In [238]:
# タイトルで両方判定できたもの
df_c.loc[(~df_c.gender_title.isna()) & (~df_c.age_title.isna())][['gender_title', 'age_title', 'title']].head(10)

,gender_title,age_title,title
47,M,16.0,AITA Am I (16m) the asshole for calling my dad (44m) fat for skinny shaming me?
617,F,33.0,AITA I (33f) changed my mind about moving to another town with my bf (37m)
618,F,28.0,"AITA for going to a ""third world"" country to see my (28F) boyfriend (29M)?"
621,F,21.0,WIBTA if I (21F) make my brother (29M) pay for my attire to his wedding?
715,F,21.0,AITA for telling my (F21) BF (M22) that the school he goes to will not fulfill him?
719,F,14.0,AITA for enforcing a family agreement to make our daughter [14f] clean up after our pets for a week?
918,M,26.0,AITA My ex 25F expects me 26M to pay off her credit card(s)
920,F,21.0,AITA || My Roommates(F22) Cats Broke My(F21) Laptop
988,M,26.0,AITA for being mad my (26M) gf (27F) sleeps out of doing housework?
1018,M,25.0,"AITA for not wanting to go to a friend's (25F) birthday ""party"" because her boyfriend (26M) hates me(25M)?"


In [239]:
# タイトルで片方だけ判定できたもの
df_c.loc[(~df_c.gender_title.isna()) | (~df_c.age_title.isna())][['gender_title', 'age_title', 'title']].head(10)

,gender_title,age_title,title
0,F,NaN,AITA for embarrassing my husband in front of his friends?
18,M,NaN,AITA for asking my wife to watch our child?
39,M,NaN,WIBTA if I take vacation without my wife
46,F,NaN,AITA for not giving any of the money my husband left for our children to his mother?
47,M,16.0,AITA Am I (16m) the asshole for calling my dad (44m) fat for skinny shaming me?
49,F,NaN,AITA for wanting to know what’s up with my bf and his parents?
56,F,NaN,AITA for asking my boyfriend to stop chewing with his mouth open?
57,F,NaN,AITA for yelling at my husband after he put me on blast in front of his parents?
70,F,NaN,"AITA, I made a joke, hurt my boyfriend's feelings and he won't trust me anymore"
73,M,NaN,AITA for going to a famous girl's house without being explicit about it with my girlfriend? (I know how this will probably sound and go)


In [240]:
# タイトルでどちらも判定できなかったもの
df_c.loc[(df_c.gender_title.isna()) & (df_c.age_title.isna())][['gender_title', 'age_title', 'title']].head(10)

,gender_title,age_title,title
1,None,NaN,AITA from eating a cookie
2,None,NaN,AITA for yelling at my roommate to stop touching my stuff.
3,None,NaN,WIBTA to play an April Fool’s prank on my brother?
4,None,NaN,AITA for joking with my sister in front of her girlfriend?
5,None,NaN,AITA for asking my friend’s potential date to prom
6,None,NaN,AITA Who does not want to help pay for her cousin's bridal shower?
7,None,NaN,AITA for being “nonchalant” about my mom’s passing to my mother’s side of the family?
8,None,NaN,AITA for telling my 13 y/o niece and her 3 friends that they have a 9pm “lights out” because they have a game tomorrow? Apparently I am evil.
9,None,NaN,AITA For not returning to China to visit my grandparents.
10,None,NaN,AITA for wanting to drink soda and breaking my penance?


In [241]:
# 本文でどちらも判定できたもの
df_c.loc[(~df_c.gender.isna()) & (~df_c.age.isna())][['gender', 'age', 'selftext']].head(10)

,gender,age,selftext
0,F,26.0,"I (26 F) am married to K (25 M) , i met him a few year back when my best friend and his best friend started dating, after that our friend groups kind of joined together. So all of my husband’s friends are also my friends and all my friends are his friends.\n\nI’ve know my best friend (A) for 15 years, we’ve always been extremely close and lived together at one point, she’s literally part of my family, she comes to all of my family events, my siblings refer to her as their sister and she just been extremely involved in my life for years.\nNow, me and my husband have a 6 month old daughter. My dad is extremely wealthy so he helps us out financially until my daughter is old enough to go to school, then i will take a job at his company that pays well. My husband has a job that he works from 7am - 4 pm and then afterwards him and his friends often hangout at my house or at one of their house’s. \n\nHere’s the problem, my husband’s best friend cheated on my best friend a few weeks ago, she’s completely heartbroken and since they lived together my husband and i agreed that it’s okay that she stays with us until they work things out. She’s not really up to seeing him just yet. A is a huge help with the baby, she’s so good with her and the baby loves her. When my husband and his friends get home they normally talk until 8 pm ish so he doesn’t help with the baby too much when he gets home, which i don’t mind it’s just nice to have a little help. Yesterday night my husband opened the front door , peaked his head in, looked right at me and A , rolled his eyes and walked out. Him and his friends came in a few moments later and were all being cold and rude to A all evening. After an hour or two my husband walks right up to me and A and says “does she always have to be here? I want to bring (friends name) here tonight.” And rolled his eyes, i responded “if i want to bring my best friend into the house my dad paid for, i can, and if i choose to have somebody help me with the baby that you’re not looking after, it’s going to be her.” His friends just sat there silently and he just walked away and sat back down . He’s giving me the silent treatment and A left and is staying with her mom as she feels uncomfortable being in the house now. So AITA?"
2,F,20.0,"My roommate (about to be 27 F) and me (20F) have been living together for almost 6 months now. I’ve been getting really frustrated recently, because everything in the apartment is constantly getting moved around, and its annoying to constantly look for my things, including my server book, my work shoes, and my ipad to finish projects. BASICALLY EVERYTHING IMPORTANT TO ME RN TO MAKE MONEY. Come to find out, it’s because my roommate is constantly touching my stuff and moving it around to clean. To which I understand, but usually people move stuff back to their normal spots, if it isn’t THEIR OWN STUFF. So, I tried talking to her about it, but like any sit-down convo with her, she thinks I’m personally attacking her. Ive tried multiple ways to talk to her about it and it just comes out to the same conclusion. In her words, “ You’re the child and you need to shut the fuck up”. When she said that, I lost it. I told her, “ At least I’m not touching other people shit and then blaming it on other people!”, which made her upset and cry to our neighbors, who are now telling me to apologize. \nI will apologize, if it means a better friendship in the end. But guys, I’ve genuinely tried every single way possible. AITA?\n\n\n\nQUICK EDIT: Just to lyk i basically live in the living room area. but my space is very well defined ( basically yk what part is bedroom and what part is the living room) Ive very OCD about my stuff and keep very tidy. I don’t leave anything in her way or anything like that. I’m a people pleaser and I try not to make anyone upset or uncomfortable in their own home. I leave my important things on my nightstand, thats away from the living room area."
4,M,3

In [242]:
# 本文で片方だけ判定できたもの
df_c.loc[(~df_c.gender.isna()) | (~df_c.age.isna())][['gender', 'age', 'selftext']].head(10)

,gender,age,selftext
0,F,26.0,"I (26 F) am married to K (25 M) , i met him a few year back when my best friend and his best friend started dating, after that our friend groups kind of joined together. So all of my husband’s friends are also my friends and all my friends are his friends.\n\nI’ve know my best friend (A) for 15 years, we’ve always been extremely close and lived together at one point, she’s literally part of my family, she comes to all of my family events, my siblings refer to her as their sister and she just been extremely involved in my life for years.\nNow, me and my husband have a 6 month old daughter. My dad is extremely wealthy so he helps us out financially until my daughter is old enough to go to school, then i will take a job at his company that pays well. My husband has a job that he works from 7am - 4 pm and then afterwards him and his friends often hangout at my house or at one of their house’s. \n\nHere’s the problem, my husband’s best friend cheated on my best friend a few weeks ago, she’s completely heartbroken and since they lived together my husband and i agreed that it’s okay that she stays with us until they work things out. She’s not really up to seeing him just yet. A is a huge help with the baby, she’s so good with her and the baby loves her. When my husband and his friends get home they normally talk until 8 pm ish so he doesn’t help with the baby too much when he gets home, which i don’t mind it’s just nice to have a little help. Yesterday night my husband opened the front door , peaked his head in, looked right at me and A , rolled his eyes and walked out. Him and his friends came in a few moments later and were all being cold and rude to A all evening. After an hour or two my husband walks right up to me and A and says “does she always have to be here? I want to bring (friends name) here tonight.” And rolled his eyes, i responded “if i want to bring my best friend into the house my dad paid for, i can, and if i choose to have somebody help me with the baby that you’re not looking after, it’s going to be her.” His friends just sat there silently and he just walked away and sat back down . He’s giving me the silent treatment and A left and is staying with her mom as she feels uncomfortable being in the house now. So AITA?"
1,None,16.0,"Story is just as stupid as the title.\n\nSo my sister (25) bough cookies home last night. They were $5 for a 8 pack. Well she didn't say that they weren't for everyone amd since she frequently goes grocery shopping for my mom I thought they were for everyone so I ate one today. She goes and asks my little sister (14) if she had them, she says no so my sister comes in my room while I'm trying to sleep (the box for the cookies were in my trash can because there was one left and I'm not dirtying a dish if i don't have to. Well she sees it amd takes it out of the trash and throws at me which woke me up. She tells me that she wants me to pay for the whole box because her kid is sad she didn't get the last cookie. I said that I'd pay for the single cookie I took she refused. So she goes and says she's not leaving my room until I give her $5 so I call my mom and my mom tells her to get out of my room. She ends up saying that she'll wait until I go to school on monday (I'm 16 so still in hs) so she can steal my stuff to pay for the box (she's done this before when I didn't know that the ice cream in the freezer was hers and she stole my $100 hair straighter that I had to break into her room to get back) I tell her to get out again and she does. But not before she goes and kicks my cats food bowl and threatens to dump her litter box out.\n\nAnd earlier today she was also complaining about my other older sister (21) getting mad at her for eating a donut that she bought and she thought they were for everyone. So I called her a hypocrite \n\nSo reddit aita for eating a cookie?"
2,F,20.0,"My roommate (about to be 27 F) and me (20F) have been living together for almost 6 m

In [243]:
# 本文でどちらも判定できなかったもの
df_c.loc[(df_c.gender.isna()) & (df_c.age.isna())][['gender', 'age', 'selftext']].head(10)

,gender,age,selftext
3,None,NaN,"So as you all know, tomorrow is April Fool’s Day. Both my brother and I won’t be home (he’s at a sleepover and I’m heading over to go babysitting), which is why I can’t ask him. He doesn’t have a cell, so I can’t call or text either.\n\nWe planned a “prank” together (its just a version of the same prank we do every year). This year we are dressing as each other. He would be dressing fancy/paying more attention to his outfit. I would be mismatching with a graphic tee. It’s nice.\n\nHowever, I thought that I could do a kind of “double-cross”. I would prank him by dressing like I normally would. It would be him dressing as me and me as me. I thought it might be fun, but at the same time I don’t want him to be mad at me.\n\nI would just do it if we were both at home, and then I could change into his clothes. But I can’t. We’ll both be stuck in whatever clothes we brought. Especially since I babysit from 05:30 til 22:00. Without going home.\n\nAges are 15f (almost 16!) and 13m. This is all fun, even if he did get mad he’d be over it in an hour. Please don’t start making any assumptions like a lot of AITA people do."
7,None,NaN,"I’m 24(F) my mom passed 4 days after giving birth to me and my twin brother. We were raised by our dad who gave us the best life. We grew up with our mothers side of the family as well. Who never let us forget our mother and always was there if we had questions about her. \n\nEvery Year on the passing of her death our whole mothers side including my brother and I would go to her grave 3 days before our birthday. Of Course, I would go every single year. \n\nI use to cry about it when I was younger because I wanted a mom, but as I got older I cried less and less just naturally moved on about my life it was hard for my brother because he always wanted a mom he was depressed a lot and I was there for him but we didn’t share the same feelings because in my eyes at least I had 1 parent.\n\nFast forward now our birthday is coming up in May and my friends and I booked a trip to Jamaica and the days lined up with the date my mother passed on. I was fine with it because I thought “What’s one year to miss?” My Mothers side unfortunately wasn’t okay with it and blew the whole situation out of proportion in my opinion.\n\nMy Aunt and Grandma were basically verbally attacking me saying “I don’t care about her” & “She’s probably turning over in her grave listening to this” and “She would probably be ashamed to have a daughter like me” what got me was “How could you not care about your own mom as much love that she had for you” I blew up.\n\nI remember exactly what I said word from word crying and screaming at them. I was saying “You don’t think I care about her I always wanted a mom.. every single time when i was little seeing a mother and her daughter our shopping I cried. Not having a mother to teach me about a period, doing my hair, talking about boys. I LITERALLY WANTED ALL OF THAT. But at the same time your feelings about her are different from mom’s you guys both knew my mom y’all watched her grow watched her do everything have endless memories with her I CAN’T GRIEVE NOBODY I NEVER KNEW.” I broke down and just walked out the house and left after that. \n\nSince then both my aunt and grandma told me I was disrespectful and it’s still rude for me not to come completely disregarding my feelings and everything I said. The only people that understands me is my twin, dad and grandfather. AITA? \n\nP.S.. I’m still going on my trip."
8,None,NaN,"I’m 27 and watching my niece because my sister got called away on business. My sister’s parting words to me were “they have a game tomorrow, please make sure they get in bed on time.” When my niece showed up this afternoon I told she and her three friends that they need to be lights out by 9 PM. They have been sulking ever since and my niece literally told me I’m the evil aunt and she never wants to speak to me again.\n\nI asked her what she was plan

In [244]:
# 本文になかった場合はタイトルから取る
df_c.loc[df_c.gender.isnull(), 'gender'] = df_c.gender_title
df_c.loc[df_c.age.isnull(), 'age'] = df_c.age_title

In [245]:
# 判定できた割合
df_c[['age', 'gender']].notnull().sum() / df_c.shape[0] * 100.00

age       45.948094
gender    58.766919
dtype: float64

In [247]:
df_c[['link_flair_text', 'num_comments', 'over_18', 'score', 'url',
       'selftext', 'title', 'id', 'edited', 'is_self', 'permalink', 'downs',
       'ups', 'created', 'date', 'age', 'gender']].to_parquet("labeled_post_genage.parquet")